In [1]:
import importlib_resources as resources
import sys
import pandas as pd
import threading
from datetime import datetime, timedelta
import os

from etherscan import Etherscan
import json
import time
import requests
from hexbytes import HexBytes
from web3 import Web3
import flatdict
from datetime import datetime

import mysql.connector

# 1.Load Data

## 1.1 Load healthy Trx csv

In [2]:
df_healthy_trx = pd.read_csv('agg_healthy_trx.csv')

In [3]:
df_healthy_trx.head()

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract
0,0,0,0xb418ACF9138E5D00D6a249423841F993514A05C1,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.019502,2021-08-18 14:38:08,0x5a452513330ae622cea05a7157b9cf3ab7cadd293440...,2.100000e-15,4.971000e-09,1,0x,13049794,True
1,1,1,0xd4CeAB765074ee15A7fBB8201E705077088d6edF,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.029545,2021-08-18 14:38:46,0xc8d1300a2f9b08ddc7e19a778d4057d47db81ae4892b...,2.100000e-15,4.981000e-09,1,0x,13049799,True
2,2,2,0x401A7D2dF89b8855783Ee12fe055a58CE95DD3EC,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.004105,2021-08-18 14:43:02,0x49abe867f3ac91c2b857072feedbad7731425e66784e...,2.100000e-15,4.706000e-09,1,0x,13049826,True
3,3,3,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xa25Cc31F54fD90BBa0d653E936d78F9ca137723f,0.003000,2021-08-18 14:43:10,0x72c60a7137a3be23f0d2669e5d080c21a4217b7d9b9a...,2.100000e-15,4.701000e-09,1,0x,13049828,True
4,4,4,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.000000,2021-08-18 14:46:18,0x18221f69e7bfdf02453db20edf314a0a766a92446ec9...,1.000000e-14,4.483167e-09,1,0xa9059cbb,13049845,True


## 1.2 Load Healthy Wallets:

In [4]:
df_healthy_wallets = pd.read_csv('healthy_wallets.csv')
df_healthy_wallets.head()

,Unnamed: 0.1,Unnamed: 0,healthy_wallets,Block,is_contract
0,0,0,0x0000000000007F150Bd6f54c40A34d7C3d5e9f56,15941096.0,True
1,1,1,0x000000000000Ad05Ccc4F10045630fb830B95127,15898566.0,True
2,2,2,0x00000000000A47b1298f18Cf67de547bbE0D723F,14613635.0,True
3,3,3,0x0000000000304a767881fDcCB30FCEB51f6221e2,15833199.0,True
4,4,4,0x00000000006c3852cbEf3e08E8dF289169EdE581,16045486.0,True


In [5]:
df_healthy_trx[df_healthy_trx.Trx_Amount==0].head(3)

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract
4,4,4,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.0,2021-08-18 14:46:18,0x18221f69e7bfdf02453db20edf314a0a766a92446ec9...,1.000000e-14,4.483167e-09,1,0xa9059cbb,13049845,True
6,6,6,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.0,2021-08-18 14:49:00,0xf49b9164983adae724e05b06ca157d73f5e99464160c...,5.168800e-15,5.000000e-09,1,0x23b872dd,13049861,True
8,8,8,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.0,2021-08-18 14:50:36,0xffdc00adfc5f4633c3a673006ec4553b7f2d8ad88084...,5.171200e-15,5.000000e-09,1,0x23b872dd,13049868,True


## 1.3 Fix Data sources Address

In [6]:
nodeURL = 'https://mainnet.infura.io/v3/79961c6d4d314c1fb1acae4f89d42cdc' # Trustly

# Load in node to access the blockchain and connect to the network
web3 = Web3(Web3.HTTPProvider(nodeURL))

### 1.3.1 Flagged Contract Addr:

In [9]:
def wallet_is_contract(wallet_addr):
    result = Web3.toChecksumAddress(wallet_addr)
    if len(web3.eth.getCode(result)) == 0:
        return False
    else:
        return True

df_healthy_wallets['is_contract'] = df_healthy_wallets['healthy_wallets'].apply(lambda x: wallet_is_contract(x))

KeyboardInterrupt: 

In [8]:
len(df_healthy_wallets)

9477

In [9]:
len(df_healthy_wallets[df_healthy_wallets.is_contract==False])

9060

In [10]:
df_healthy_wallets[df_healthy_wallets.healthy_wallets== Web3.toChecksumAddress('0x292f04a44506c2fd49bac032e1ca148c35a478c8')]

,Unnamed: 0.1,Unnamed: 0,healthy_wallets,Block,is_contract
1480,1480,1480,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,15941096.0,False


### 1.3.2 Fix Addr in Healthy Trx:

In [53]:
def fix_addr(wallet_addr):
    if wallet_addr != None and len(str(wallet_addr)) > 5:
        hex_wallet_addr = Web3.toChecksumAddress(wallet_addr)
        if hex_wallet_addr != None:
            return hex_wallet_addr
    else:
        return wallet_addr

fix_addr('0x')

'0x'

In [50]:
df_healthy_trx['Trx_From_Wallet'] = df_healthy_trx['Trx_From_Wallet'].apply(lambda x: fix_addr(x))

In [54]:
df_healthy_trx['Trx_To_Wallet'] = df_healthy_trx['Trx_To_Wallet'].apply(lambda x: fix_addr(x))

In [12]:
df_healthy_trx.head()

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract
0,0,0,0xb418ACF9138E5D00D6a249423841F993514A05C1,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.019502,2021-08-18 14:38:08,0x5a452513330ae622cea05a7157b9cf3ab7cadd293440...,2.100000e-15,4.971000e-09,1,0x,13049794,True
1,1,1,0xd4CeAB765074ee15A7fBB8201E705077088d6edF,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.029545,2021-08-18 14:38:46,0xc8d1300a2f9b08ddc7e19a778d4057d47db81ae4892b...,2.100000e-15,4.981000e-09,1,0x,13049799,True
2,2,2,0x401A7D2dF89b8855783Ee12fe055a58CE95DD3EC,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.004105,2021-08-18 14:43:02,0x49abe867f3ac91c2b857072feedbad7731425e66784e...,2.100000e-15,4.706000e-09,1,0x,13049826,True
3,3,3,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xa25Cc31F54fD90BBa0d653E936d78F9ca137723f,0.003000,2021-08-18 14:43:10,0x72c60a7137a3be23f0d2669e5d080c21a4217b7d9b9a...,2.100000e-15,4.701000e-09,1,0x,13049828,True
4,4,4,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.000000,2021-08-18 14:46:18,0x18221f69e7bfdf02453db20edf314a0a766a92446ec9...,1.000000e-14,4.483167e-09,1,0xa9059cbb,13049845,True


### 1.3.3 Validations:

In [13]:
df_healthy_trx[df_healthy_trx['Trx_From_Wallet']==str('0x00131437c080f9a32ED4B4959c2c0Eb5e877bBBC').lower()]

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract


### 1.3.4 Save new healthy trx

In [56]:
#df_healthy_trx.to_csv('agg_healthy_trx.csv')

# 2. Gathering Contract Data for Healthy Wallets

In [14]:
df_healthy_trx = pd.read_csv('agg_healthy_trx.csv')
df_healthy_trx.head()

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract
0,0,0,0xb418ACF9138E5D00D6a249423841F993514A05C1,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.019502,2021-08-18 14:38:08,0x5a452513330ae622cea05a7157b9cf3ab7cadd293440...,2.100000e-15,4.971000e-09,1,0x,13049794,True
1,1,1,0xd4CeAB765074ee15A7fBB8201E705077088d6edF,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.029545,2021-08-18 14:38:46,0xc8d1300a2f9b08ddc7e19a778d4057d47db81ae4892b...,2.100000e-15,4.981000e-09,1,0x,13049799,True
2,2,2,0x401A7D2dF89b8855783Ee12fe055a58CE95DD3EC,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.004105,2021-08-18 14:43:02,0x49abe867f3ac91c2b857072feedbad7731425e66784e...,2.100000e-15,4.706000e-09,1,0x,13049826,True
3,3,3,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xa25Cc31F54fD90BBa0d653E936d78F9ca137723f,0.003000,2021-08-18 14:43:10,0x72c60a7137a3be23f0d2669e5d080c21a4217b7d9b9a...,2.100000e-15,4.701000e-09,1,0x,13049828,True
4,4,4,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.000000,2021-08-18 14:46:18,0x18221f69e7bfdf02453db20edf314a0a766a92446ec9...,1.000000e-14,4.483167e-09,1,0xa9059cbb,13049845,True


In [15]:
df_healthy_wallets = pd.read_csv('healthy_wallets.csv')
df_healthy_wallets.head()

,Unnamed: 0.1,Unnamed: 0,healthy_wallets,Block,is_contract
0,0,0,0x0000000000007F150Bd6f54c40A34d7C3d5e9f56,15941096.0,True
1,1,1,0x000000000000Ad05Ccc4F10045630fb830B95127,15898566.0,True
2,2,2,0x00000000000A47b1298f18Cf67de547bbE0D723F,14613635.0,True
3,3,3,0x0000000000304a767881fDcCB30FCEB51f6221e2,15833199.0,True
4,4,4,0x00000000006c3852cbEf3e08E8dF289169EdE581,16045486.0,True


In [16]:
df_healthy_wallets = df_healthy_wallets[df_healthy_wallets.is_contract==False]

In [17]:
#Wallet_Address = list(walletBlocked_dic)[244:]
Wallet_Address = list(df_healthy_wallets.healthy_wallets)

#last processed Wallet: 0x6190ed7BCdeFD9aC9F3357510c5d6bfC4b218B75
#Wallet_Address = Wallet_Address[:Wallet_Address.index('0x6190ed7BCdeFD9aC9F3357510c5d6bfC4b218B75')]
len(Wallet_Address)

9060

In [18]:
# Load ABIs and Interface IDs
contractABIs = json.load(open('./contractABIs.json')) # Loads the ABIs from the JSON file

# Constants to identify ERC20 Transfer and Approval events/transactions
TRANSFER_EVENT_HASH = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
APPROVAL_EVENT_HASH = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"

# Function to get ERC20 Information from a transaction
def verify_erc20(address: str) -> dict:
    """ Returns a map with the contract's properties if it's ERC20, else raise an error """

    # Load in contract with ERC20 ABI to use ERC20 functions
    contract = web3.eth.contract(address = address, abi = contractABIs["ERC20"])

    # Functions
    token = {
        "totalSupply" : contract.functions.totalSupply().call(),
        "symbol" : contract.functions.symbol().call(),
        "name" : contract.functions.name().call(),
        "decimals" : contract.functions.decimals().call()
    }

    return token # Returns token properties in a dictionary

# Function to get all of the transaction's relevant information
def get_transaction_info(address: str) -> dict:
    """ Returns a map with the address' relevant transaction information"""

    transactionInfo = {} # A dictionary with all of the transactions' information

    transaction = web3.eth.get_transaction(address) # Get all of the transaction information

    block = web3.eth.get_block(transaction["blockHash"]) # Get the block information

    receipt = web3.eth.get_transaction_receipt(address) # Get the receipt information

    # Addresses of all contracts created by the transaction TODO: ADD THIS FEATURE - Web3.py Latest Build Only
    # transactionInfo["createdContracts"] = transaction["creates"]
    
    # Transaction Hash
    transactionInfo["hash"] = transaction["hash"].hex()

    # Block Number
    transactionInfo["blockNumber"] = transaction["blockNumber"]

    # Transaction Timestamp
    transactionInfo["timestamp"] = block["timestamp"]

    # Detect if the receiver is a contract
    if "to" in transaction:
        if transaction["to"] != None:
            transactionInfo["receiverIsContract"] = web3.eth.getCode(transaction["to"]) != "0x"

    # Transaction Sender and Receiver
    transactionInfo["sender"] = transaction["from"]
    transactionInfo["receiver"] = transaction["to"] # Whom the transaction is sent to

    # Transaction Type - Can be ERC20 or Other. If It's ERC20 a tokenAddress will be added    
    try:
        token = verify_erc20(transaction["to"]) # Confirms it's ERC20
        transactionInfo["ERC20Info"] = {
            "eventInfo": None, # If transaction emits an event (Transfer or Approval)
            "tokenAddress": transaction["to"], # Pass the token address
            "tokenDetails": token # Information about the token
        }

        # Find Transfer Event and get associated value
        for log in receipt["logs"]:
            if HexBytes(TRANSFER_EVENT_HASH) in log["topics"]: # If the Transfer() event is found:
                transactionInfo["ERC20Info"]["eventInfo"] = {
                    "type": "transfer",
                    "transferReceiver": Web3.toChecksumAddress("0x" + log["topics"][2].hex()[26:]), # The address of the token's receiver
                    "tokenQuantity": int(log["data"], 16), # The amount of token transfered
                    "transferReceiverIsContract": web3.eth.getCode(Web3.toChecksumAddress("0x" + log["topics"][2].hex()[26:])) != "0x" # Whether the transfer receiver is a contract
                }

                break
            elif HexBytes(APPROVAL_EVENT_HASH) in log["topics"]: # If the Approval() event is found:
                transactionInfo["ERC20Info"]["eventInfo"] = {
                    "type": "approval",
                    "addressApproved": Web3.toChecksumAddress("0x" + log["topics"][1].hex()[26:]), # The address of the approved spender
                    "tokenAllowance": int(log["data"], 16), # The amount of tokens the spender can use
                    "approvedSpenderIsContract": web3.eth.getCode(Web3.toChecksumAddress("0x" + log["topics"][1].hex()[26:])) != "0x" # Whether the transfer receiver is a contract
                }
                break  
        
    except Exception: # If it's not an ERC20
        transactionInfo["ERC20Info"] = None

    # Ethereum Value
    transactionInfo["ethValue"] = transaction["value"] / (10 ** 18) # Convert to ETH

    return transactionInfo


## 2.2 Final version: update smart contract info

In [20]:
df = df_healthy_trx.copy()
df.head()

,Unnamed: 0.1,Unnamed: 0,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID,BlockNumber,receiverIsContract
0,0,0,0xb418ACF9138E5D00D6a249423841F993514A05C1,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.019502,2021-08-18 14:38:08,0x5a452513330ae622cea05a7157b9cf3ab7cadd293440...,2.100000e-15,4.971000e-09,1,0x,13049794,True
1,1,1,0xd4CeAB765074ee15A7fBB8201E705077088d6edF,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.029545,2021-08-18 14:38:46,0xc8d1300a2f9b08ddc7e19a778d4057d47db81ae4892b...,2.100000e-15,4.981000e-09,1,0x,13049799,True
2,2,2,0x401A7D2dF89b8855783Ee12fe055a58CE95DD3EC,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0.004105,2021-08-18 14:43:02,0x49abe867f3ac91c2b857072feedbad7731425e66784e...,2.100000e-15,4.706000e-09,1,0x,13049826,True
3,3,3,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xa25Cc31F54fD90BBa0d653E936d78F9ca137723f,0.003000,2021-08-18 14:43:10,0x72c60a7137a3be23f0d2669e5d080c21a4217b7d9b9a...,2.100000e-15,4.701000e-09,1,0x,13049828,True
4,4,4,0x292f04a44506c2fd49Bac032E1ca148C35A478c8,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.000000,2021-08-18 14:46:18,0x18221f69e7bfdf02453db20edf314a0a766a92446ec9...,1.000000e-14,4.483167e-09,1,0xa9059cbb,13049845,True


In [21]:
df = df[(df['receiverIsContract']==True)]

In [24]:
from eth_api_functions import lint_transaction_list
from tqdm import tqdm
import numpy as np
import itertools

list_nodeURL = (["https://mainnet.infura.io/v3/181eb28be5e143428470f756610ffbfb", #madinfura3@gmail.com
                 "https://mainnet.infura.io/v3/97fb459c3aa449b2a450469bbfb21e34", #outlook novaims
                 "https://mainnet.infura.io/v3/b0a609294ac24ce1b4552134b4410840",
                 "https://mainnet.infura.io/v3/4a14058aa8a5482daf372cca777a1495",
                 "https://mainnet.infura.io/v3/1a0ee6cc96904f5b9212da02f89b0a6c", #yahoo 1st
                 "https://mainnet.infura.io/v3/7574591d58704e9a86ddf0806b48a0e8",
                 "https://mainnet.infura.io/v3/3ce87e1a765843ad8c222b209d0ffc0e",
                 "https://mainnet.infura.io/v3/4b98031d11854b0aa0e1c1e2a406d43e", #madinfura@gmail.com
                 "https://mainnet.infura.io/v3/0c7b7e45c3fb436dae3ae7906004e57a", #jfinfura@gmail.com
                 "https://mainnet.infura.io/v3/87b3825246cd45549daeaeb7b66f033b", #jeinfura@gmail.com
                 "https://mainnet.infura.io/v3/e48f482d43524ccaa3c225fd261512f3",
                 "https://mainnet.infura.io/v3/2ba0cf55a54e4af2b4819b53fd8ecdee",
                 "https://mainnet.infura.io/v3/79961c6d4d314c1fb1acae4f89d42cdc",
                 "https://mainnet.infura.io/v3/a7305897130047e486e21012c96e5877",
                 "https://mainnet.infura.io/v3/4a7d054e48fd4ea2ac1ceac733c3fe9d",
                 "https://mainnet.infura.io/v3/b30a4bab49f64477b0eb894978628700", #jlinfura@gmail.com
                 "https://mainnet.infura.io/v3/f4601daf8fe44492b6a71088b3511dbe", #jhinfura@gmail.com
                 "https://mainnet.infura.io/v3/5b573a6b36ac4185b3d55c968fc40c1c"])#jginfura@gmail.com  #"https://mainnet.infura.io/v3/921f492428a6498a85b3c02dc2d48576",

nodeURL_size = len(list_nodeURL)

n = 5000
    
#############################################
###    Get all relevant Transactions.     ###
#############################################
wllt_trx = dict()
with tqdm(total=len(Wallet_Address)) as pbar:
    for walletAddress in Wallet_Address:

        txnHashList = list(df[(df['Trx_To_Wallet'] == Web3.toChecksumAddress(walletAddress))
                              | (df['Trx_From_Wallet'] == Web3.toChecksumAddress(walletAddress))]['Trx_Hash'])

        wllt_trx[walletAddress] = txnHashList
        pbar.update(1)

loop_size = sum(len(v) for v in wllt_trx.values())

############ create List of trx to iterate: ############
trx_main_list = list(wllt_trx.values())
flat_list = [item for sublist in trx_main_list for item in sublist]

#AUX to remove
#flat_list = flat_list[379000:]
flat_list = flat_list[1036000:]
loop_size = len(flat_list)


all_wllts_trx_lst = [flat_list[i:i + n] for i in np.arange(0, len(flat_list), n)]

tpl_trx_nodeURL = [list(itertools.chain(i)) for i in zip(all_wllts_trx_lst, list_nodeURL*(int(len(all_wllts_trx_lst)/nodeURL_size)+1))]

100%|██████████| 9060/9060 [40:58<00:00,  3.68it/s]  


In [37]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from eth_api_functions import lint_transaction_list
from tqdm import tqdm
import numpy as np
import itertools

def rotate(l, n):
    return l[n:] + l[:n]


### VAR DEFINITIONS ###
#declare empty dataframes:
#ETHEREUM_TRANSFERS = pd.DataFrame()
#ERC20_TRANSFERS = pd.DataFrame()
#ERC20_APPROVALs = pd.DataFrame()
#OTHER_TRANSACTIONS = pd.DataFrame()


bucket_n = 323
### process all trx with a step of n for each infura Key: ###
with tqdm(total=loop_size) as pbar:
    for trx_bucket in tpl_trx_nodeURL:

        ### VAR DEFINITIONS ###
        #declare empty dataframes:
        ERC20_TRANSFERS = pd.DataFrame(data=None, columns=ERC20_TRANSFERS.columns)
        ERC20_APPROVALs = pd.DataFrame(data=None, columns=ERC20_APPROVALs.columns)
        OTHER_TRANSACTIONS = pd.DataFrame(data=None, columns=OTHER_TRANSACTIONS.columns)
        
        web3 = Web3(Web3.HTTPProvider(trx_bucket[1]))
        
        # Runs through txnHashList and fetches the info of those transactions to the blockchain
        processedTxnList = []
        bucket_n += 1
        txnHashList = trx_bucket[0]
        for txn in txnHashList:
            pbar.update(1)
            txn_ = txn
            t = get_transaction_info(txn)
            processedTxnList.append(t)

        #Lint all transactions into 4 different types: Ethereum Transfers, ERC20 Transfers, ERC20 Approvals and Others
        lintedList = lint_transaction_list(processedTxnList)

        #ethList = lintedList["eth"]
        erc20TransferList = lintedList["erc20Transfer"]
        erc20ApprovalList = lintedList["erc20Approval"]
        otherList = lintedList["other"]

        # TO DO: Create 4 tables and add results of each list to each table

        tableNames = [
            #"ETHEREUM_TRANSFERS",
            "ERC20_TRANSFERS",
            "ERC20_APPROVALs",
            "OTHER_TRANSACTIONS"
        ]

        # engine = connect_to_db('admin', 'wWusLXWEsxNqaviwGPsP', 'cryptologic_BE_Dev')

        # Run through all lists and add them to their respective tables
        i = 0
        for lst in (erc20TransferList, erc20ApprovalList, otherList):
            tableName = tableNames[i]

            for txn in lst:
                flattenedTxn = flatdict.FlatDict(txn, delimiter='_')

                # Transform all values to lower case
                for key in flattenedTxn:
                    if type(flattenedTxn[key]) == str:
                        flattenedTxn[key] = flattenedTxn[key].lower()
                    flattenedTxn[key] = [flattenedTxn[key]]

                output = pd.DataFrame(dict(flattenedTxn)) # Must be converted to dict from FlatDict

                if tableName == "ETHEREUM_TRANSFERS":
                    #ETHEREUM_TRANSFERS = output.append(ETHEREUM_TRANSFERS)
                    continue
                if tableName ==  "ERC20_TRANSFERS":
                    ERC20_TRANSFERS = output.append(ERC20_TRANSFERS)
                if tableName == "ERC20_APPROVALs":
                    ERC20_APPROVALs = output.append(ERC20_APPROVALs)
                if tableName == "OTHER_TRANSACTIONS":
                    OTHER_TRANSACTIONS = output.append(OTHER_TRANSACTIONS)


            i += 1

        ERC20_TRANSFERS.to_csv(f'ERC20_TRANSFERS_b_{bucket_n}.csv',mode='w+')
        ERC20_APPROVALs.to_csv(f'ERC20_APPROVALs_b_{bucket_n}.csv',mode='w+')
        OTHER_TRANSACTIONS.to_csv(f'OTHER_TRANSACTIONS_b_{bucket_n}.csv',mode='w+')

        f = open("Notes_dataset_gen.txt", "a")
        f.write("Transaction: " + str(txn_) + " - Processed with success! | Iteration: " + str(bucket_n) + " | Timestamp: " + str(datetime.now()) + "\n")
        f.close()


  3%|▎         | 30000/928975 [5:36:02<145:08:48,  1.72it/s]

# 3. AUX loop smart contracts

In [23]:
len(txnHashList)

10607

In [22]:
txnHashList.index('0x91083906ecc2ac657a05c6a5c6ae947893e81feceab0235326d1a4db75b63e2c')

6163

In [24]:
txnHashList[6163:]

['...',
 ...]

In [ ]:
for txn in txnHashList[6163:]:
    print('Trx Hash: ' + str(txn))
    t = get_transaction_info(txn)
    time.sleep(1)
    processedTxnList.append(t)
    
#Lint all transactions into 4 different types: Ethereum Transfers, ERC20 Transfers, ERC20 Approvals and Others
lintedList = lint_transaction_list(processedTxnList)

#ethList = lintedList["eth"]
erc20TransferList = lintedList["erc20Transfer"]
erc20ApprovalList = lintedList["erc20Approval"]
otherList = lintedList["other"]

# TO DO: Create 4 tables and add results of each list to each table

tableNames = [
    #"ETHEREUM_TRANSFERS",
    "ERC20_TRANSFERS",
    "ERC20_APPROVALs",
    "OTHER_TRANSACTIONS"
]

# engine = connect_to_db('admin', 'wWusLXWEsxNqaviwGPsP', 'cryptologic_BE_Dev')

# Run through all lists and add them to their respective tables
i = 0
for lst in (erc20TransferList, erc20ApprovalList, otherList):
    tableName = tableNames[i]

    for txn in lst:
        flattenedTxn = flatdict.FlatDict(txn, delimiter='_')

        # Transform all values to lower case
        for key in flattenedTxn:
            if type(flattenedTxn[key]) == str:
                flattenedTxn[key] = flattenedTxn[key].lower()
            flattenedTxn[key] = [flattenedTxn[key]]

        output = pd.DataFrame(dict(flattenedTxn)) # Must be converted to dict from FlatDict

        print(tableName)

        if tableName == "ETHEREUM_TRANSFERS":
            #ETHEREUM_TRANSFERS = output.append(ETHEREUM_TRANSFERS)
            continue
        if tableName ==  "ERC20_TRANSFERS":
            ERC20_TRANSFERS = output.append(ERC20_TRANSFERS)
        if tableName == "ERC20_APPROVALs":
            ERC20_APPROVALs = output.append(ERC20_APPROVALs)
        if tableName == "OTHER_TRANSACTIONS":
            OTHER_TRANSACTIONS = output.append(OTHER_TRANSACTIONS)

        #df.to_sql(tableName, con=engine, if_exists='append', index = False)

    i += 1

print('Saving to CSV files...')
#ETHEREUM_TRANSFERS.to_csv('ETHEREUM_TRANSFERS_v2.csv',mode='w+')
ERC20_TRANSFERS.to_csv('ERC20_TRANSFERS_v1.csv',mode='w+')
ERC20_APPROVALs.to_csv('ERC20_APPROVALs_v1.csv',mode='w+')
OTHER_TRANSACTIONS.to_csv('OTHER_TRANSACTIONS_v1.csv',mode='w+')

f = open("Notes_dataset_gen.txt", "a")
f.write("Wallet: " + walletAddress + " processed with success!\n Iteration: " + str(to_csv_flag))
f.close()

print("Wallet: " + walletAddress + " Ended | Iteration: " + str(to_csv_flag))

Trx Hash: 0x91083906ecc2ac657a05c6a5c6ae947893e81feceab0235326d1a4db75b63e2c
Trx Hash: 0x592fe16e0c8d6f4db0fe525c9e7b59de106222012ff484eec9cc59a655d1864e
Trx Hash: 0x22fdfd8bd475e0e2b8fd933a252d07bd9694a149f1ffff06f7a77b2cedad6b63
Trx Hash: 0xe454b4a8a81f8556a8e23b497ba020e2f86af8b3d77de86772c1f720afb522d3
Trx Hash: 0xa9a19ed235c43ce962e2121178cc6c3ec17e401579b7838a3e69d6d7af1ea435
Trx Hash: 0x47cbbce4785d073410049ca51f28209a451ed88391a7a955e172104945484ee3
Trx Hash: 0xc95f03b7f423a2d8e2f9cc1ec0e222b2cd1908f9d14b15c74bb0a63b8caf22c3
Trx Hash: 0x8ed2e1f0e3c3f809b509eb8b17c058f499a459fdfd3a7e2a03ccc27acea95cd4
Trx Hash: 0x536de867a990301b3745d3bf542f535a03580f6164be391a9fab810bcfadaa60
Trx Hash: 0x2fc55edf1916ab11e4b1fa71c9f61cfbd4d9e21509b6301ed190dcdd159f4089
Trx Hash: 0xed9f5d45648310ebd7859bab4ced6a3af4f59988443c5c33f5cffe322e17061b
Trx Hash: 0x61c79e09006f1e7e58cdb0c24ca57c76fdc9d4139a0ed2cd814a7276160a37c5
Trx Hash: 0x6359588d8b7bf8851d6da7de3f092d6f88f30a54893da4d72fc90e271be11591

In [49]:
walletAddress

'0x618fFD1cDAbeE36CE5992a857Cc7463f21272bD7'

In [ ]:
Wallet_Address = Wallet_Address[:Wallet_Address.index(walletAddress)]
len(Wallet_Address)

In [41]:
Wallet_Address.index(walletAddress)

3552

In [45]:
Wallet_Address[-2:]

['0x618c52A1c36e162aB1abA6b54182C324F7c808cA',
 '0x618fFD1cDAbeE36CE5992a857Cc7463f21272bD7']